In [1]:
# ---------------- importing EDS data files

    # - EDS data processing script developed with the help of Bram Paredis

# --- import modules

import os
import glob
import pandas as pd
import re # 're' stands for regular expressions.

# --- set working directory

base_dir = "C:/Users/u0125722/Documents/Python_Scripts/_DATA"

In [2]:
# --- look into working directory and create a list with selected files - .txt
os.chdir(base_dir + '/EDS')
txt_files = glob.glob('*.txt')

print(txt_files)
len(txt_files)

['BU04-area1_EDS.txt', 'BU04-area2_EDS.txt', 'BU04-area3_EDS.txt', 'BU04-area4_EDS.txt', 'BU04-Ore1_EDS.txt', 'BU04-Ore2over_EDS.txt', 'BU04-Ore2_EDS.txt', 'BU04-Ore3zoom_EDS.txt', 'BU04-Ore3_EDS.txt', 'BU18FA19w_area1_EDS.txt', 'BU18FA19w_area2_EDS.txt', 'BU18FA19w_area3_EDS.txt', 'BU18FA19w_area4_EDS.txt', 'BU18FA19w_area5_EDS.txt', 'BU18FA19w_area6_ore_EDS.txt', 'BU18FA19w_area7zoom_EDS.txt', 'BU18FA19w_area7_quant_EDS.txt', 'BU18FA21w-area2_EDS.txt', 'BU18FA21w-area3_EDS.txt', 'BU18FA21w-area4_EDS.txt', 'BU18FA21w-area5_EDS.txt', 'BU18FA21w-area6zoom_EDS.txt', 'BU18FA21w-area6_EDS.txt', 'BU18FA21w-area7_EDS.txt', 'BU18FA21w-area8_EDS.txt', 'BU19TS(2)-Area1b_EDS.txt', 'BU19TS(2)-Area2_EDS.txt', 'BU19TS(2)-Area4_EDS.txt', 'BU19TS(2)-Area5_EDS.txt', 'BU19TS(2)-Area5_zoom_EDS.txt', 'BU19TS(2)-Area6_EDS.txt', 'BU19TS(2)-Area7_EDS.txt', 'BU19TS(2)-Area8_EDS.txt', 'BU19TS(2)-Area9_EDS.txt', 'BU19TS(2)-Ore1_EDS.txt', 'BU19TS(2)-Ore1_zoom_EDS.txt', 'BU19TS(2)-Ore2_EDS.txt', 'BU19TS(2)-Ore3_

72

# Solution for multiple files
Applies the same code as before, but in a loop for every file in a folder

In [3]:
def EDS(i):
    # Create a dataframe from the txt file 
    df = pd.read_csv(txt_files[i], header=None, sep='\s+', names=range(30))

    # Get filename info to use for further auto df construction
    # To better order things, put this at the start of your for loop/function instead 

    filename_info = re.split("-|_", txt_files[i])


        # ----------------


    # Specify the names of the point measurements
    table_names = [f"pt{i}" for i in range(1, 30)]


    # Perform a check to see which rows in the dataframe belong to a certain
    # point measurement (called group here)
    groups = df[0].isin(table_names).cumsum()


    # Let's keep track of with how many 'point measurements' we're dealing
    n_groups = groups.unique().size


        # ----------------


    # Create a dictionary of dataframes in which the names of the point
    # measurements will be used as the dict keys and the according 
    # dataframes will be used as dict values - dict = {pt1:values}. 
    tables = {g.iloc[0,0]: g.iloc[1:].dropna(axis=1, how='all') 
              for k,g in df.groupby(groups)}


    # Only select the values we're interested in
    tables = {k: v.iloc[2:, :2].T for k,v in tables.items()}


    # Change column names to first row of df and strip '*'
    for df in tables.values():
        df.columns = df.iloc[0, :].str.strip("*")

    # Drop the now redundant row with column name info
    tables = {k: v.drop(0) for k,v in tables.items()}


        # ----------------


    df_area = pd.DataFrame()

    for k,v in tables.items():
        df_area = pd.concat((df_area, v),sort=False)


        # ----------------


    # Add additional info to df
    df_area["Analysis"] = table_names[:n_groups]
    df_area["Sample"] = filename_info[0]
    df_area["Area"] = filename_info[1]
    return df_area

In [5]:
i = 0
b = pd.DataFrame()
for file in txt_files:
    #print(file)
    a = EDS(i)
    b = pd.concat((b, a),sort=False,ignore_index = True)
    i = i + 1

b = b[["Sample", "Area", "Analysis", "F", "Na2O", "MgO", "Al2O3", "SiO2", "P2O5", "Cl", "K2O", "CaO", 
       "MnO", "FeO", "TiO2", "SrO", "Nb2O5", "BaO", "Ta2O5", "SnO2", "Yb2O3", "CuO", "ZnO", "PbO", "SO3", "UO2", 
       "ZrO2", "Au2O3", "HfO2", "MoO3", "Total"]]

#b.to_csv('out_oxides.csv', index=False)        
b

,Sample,Area,Analysis,F,Na2O,MgO,Al2O3,SiO2,P2O5,Cl,...,CuO,ZnO,PbO,SO3,UO2,ZrO2,Au2O3,HfO2,MoO3,Total
0,BU04,area1,pt1,1.83,0.92,NaN,33.40,NaN,53.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
1,BU04,area1,pt2,NaN,NaN,3.66,30.38,NaN,49.33,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
2,BU04,area1,pt3,NaN,NaN,NaN,45.24,NaN,54.76,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
3,BU04,area1,pt4,NaN,NaN,NaN,45.44,NaN,54.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
4,BU04,area1,pt5,NaN,0.61,NaN,38.08,47.59,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,BU24,Unkown,pt4,NaN,NaN,NaN,23.14,NaN,39.38,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
718,BU24,Unkown,pt5,NaN,NaN,NaN,22.78,NaN,39.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
719,BU24,Unkown,pt6,NaN,11.47,NaN,19.82,68.72,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00
720,BU24,Unkown,pt7,NaN,NaN,NaN,22.95,NaN,39.46,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00


In [7]:
c = pd.read_csv(base_dir+"/EDS/EDS_ele_to_oxides.csv", sep=',')

c

,Sample,Area,Analysis,F,Na2O,MgO,Al2O3,SiO2,P2O5,Cl,CaO,MnO,FeO,TiO2,SrO,Nb2O5,BaO,Ta2O5,Total
0,BU19TS,Area10,pt1,0.0,0.0,0.000000,0.000000,100.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100
1,BU19TS,Area10,pt2,0.0,0.0,0.000000,45.732434,0.0,54.267566,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100
2,BU19TS,Area10,pt3,0.0,0.0,0.000000,0.000000,100.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100
3,BU19TS,Area10,pt4,0.0,0.0,0.000000,0.000000,100.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100
4,BU19TS,Area10,pt5,0.0,0.0,0.000000,45.598993,0.0,54.401007,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,BU19TS,Map3,pt14,0.0,0.0,0.000000,0.000000,0.0,46.203399,0.69226,51.242162,1.862179,0.000000,0.0,0.0,0.0,0.0,0.0,100
189,BU19TS,Map3,pt15,0.0,0.0,0.000000,24.420649,0.0,40.824190,0.00000,1.466255,24.891027,8.397878,0.0,0.0,0.0,0.0,0.0,100
190,BU19TS,Map3,pt16,0.0,0.0,1.800049,30.895226,0.0,49.819284,0.00000,0.000000,0.988667,16.496774,0.0,0.0,0.0,0.0,0.0,100
191,BU19TS,Map3,pt17,0.0,0.0,1.921877,31.487535,0.0,49.571166,0.00000,0.000000,1.006063,16.013358,0.0,0.0,0.0,0.0,0.0,100


In [8]:
EDS = pd.concat((b, c),sort=False,ignore_index = True).fillna(0.0)

EDS = EDS.replace(0.0,"")

EDS.to_csv('EDS.csv', index=False) 
EDS

,Sample,Area,Analysis,F,Na2O,MgO,Al2O3,SiO2,P2O5,Cl,...,CuO,ZnO,PbO,SO3,UO2,ZrO2,Au2O3,HfO2,MoO3,Total
0,BU04,area1,pt1,1.83,0.92,,33.40,,53.25,,...,,,,,,,,,,100.00
1,BU04,area1,pt2,,,3.66,30.38,,49.33,,...,,,,,,,,,,100.00
2,BU04,area1,pt3,,,,45.24,,54.76,,...,,,,,,,,,,100.00
3,BU04,area1,pt4,,,,45.44,,54.56,,...,,,,,,,,,,100.00
4,BU04,area1,pt5,,0.61,,38.08,47.59,,,...,,,,,,,,,,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,BU19TS,Map3,pt14,,,,,,46.2034,0.69226,...,,,,,,,,,,100
911,BU19TS,Map3,pt15,,,,24.4206,,40.8242,,...,,,,,,,,,,100
912,BU19TS,Map3,pt16,,,1.80005,30.8952,,49.8193,,...,,,,,,,,,,100
913,BU19TS,Map3,pt17,,,1.92188,31.4875,,49.5712,,...,,,,,,,,,,100


In [ ]:
    # --- Things to do
    
# Classify analyses into minerals
